# Installs

In [2]:
pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/5c/01/5ab75f138bf32d7a69df61e4997e24eccad87cc009f5fb7e2a31af8a4036/torch-2.2.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/198.6 MB 5.8 MB/s eta 0:00:35
   ---------------------------------------- 0.9/198.6 MB 11.9 MB/s eta 0:00:17
   ---------------------------------------- 1.9/198.6 MB 15.0 MB/s eta 0:00:14
    --------------------------------------- 2.8/198.6 MB 16.4 MB/s eta 0:00:12
    --------------------------------------- 3.7/198.6 MB 17.0 MB/s eta 0:00:12
    --------------------------------------- 4.8/198.6 MB 18.0 MB/s eta 0:00:11
   - --------------------

In [4]:
pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/c6/75/d869f600fc33df8b8ca99943e165a4ca23b73c68dc1942098fde0a6b46f3/torchvision-0.17.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.2 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Imports

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import pickle

import pandas as pd # for writing the output csv

In [23]:
# Reproducable results
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Functions and Classes

In [28]:
# from https://github.com/keras-team/keras/blob/master/keras/utils/numerical_utils.py
# The code uses solely numpy
def to_categorical(x, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.

    E.g. for use with `categorical_crossentropy`.

    Args:
        x: Array-like with class values to be converted into a matrix
            (integers from 0 to `num_classes - 1`).
        num_classes: Total number of classes. If `None`, this would be inferred
            as `max(x) + 1`. Defaults to `None`.

    Returns:
        A binary matrix representation of the input as a NumPy array. The class
        axis is placed last.

    Example:

    >>> a = keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
    >>> print(a)
    [[1. 0. 0. 0.]
     [0. 1. 0. 0.]
     [0. 0. 1. 0.]
     [0. 0. 0. 1.]]

    >>> b = np.array([.9, .04, .03, .03,
    ...               .3, .45, .15, .13,
    ...               .04, .01, .94, .05,
    ...               .12, .21, .5, .17],
    ...               shape=[4, 4])
    >>> loss = keras.ops.categorical_crossentropy(a, b)
    >>> print(np.around(loss, 5))
    [0.10536 0.82807 0.1011  1.77196]

    >>> loss = keras.ops.categorical_crossentropy(a, a)
    >>> print(np.around(loss, 5))
    [0. 0. 0. 0.]
    """
    #if backend.is_tensor(x):
    #    return backend.nn.one_hot(x, num_classes)
    x = np.array(x, dtype="int64")
    input_shape = x.shape

    # Shrink the last dimension if the shape is (..., 1).
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])

    x = x.reshape(-1)
    if not num_classes:
        num_classes = np.max(x) + 1
    batch_size = x.shape[0]
    categorical = np.zeros((batch_size, num_classes))
    categorical[np.arange(batch_size), x] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical


class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, units, hidden_activ, activ):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, units)

        self.hidden_fc = nn.Linear(units, units)
        self.hidden_fc2 = nn.Linear(units, units)
        self.hidden_fc3 = nn.Linear(units, units)
        self.hidden_fc4 = nn.Linear(units, units)
        self.hidden_fc5 = nn.Linear(units, units)

        self.output_fc = nn.Linear(units, output_dim)
        
        self.hidden_activ = hidden_activ
        self.activ = activ

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        # x = [batch size, height * width]
        if self.hidden_activ == 'relu':
            h_1 = F.relu(self.input_fc(x))

            # h_1 = [batch size, units]

            h_2 = F.relu(self.hidden_fc(h_1))

            # h_2 = [batch size, units]

            h_3 = F.relu(self.hidden_fc2(h_2))

            # h_3 = [batch size, units]

            h_4 = F.relu(self.hidden_fc3(h_3))

            # h_4 = [batch size, units]

            h_5 = F.relu(self.hidden_fc4(h_4))

             # h_5 = [batch size, units]

            h_6 = F.relu(self.hidden_fc5(h_5))

            # h_6 = [batch size, units]
            
            
        elif self.hidden_activ == 'sigmoid':
            h_1 = F.sigmoid(self.input_fc(x))

            # h_1 = [batch size, units]

            h_2 = F.sigmoid(self.hidden_fc(h_1))

            # h_2 = [batch size, units]

            h_3 = F.sigmoid(self.hidden_fc2(h_2))

            # h_3 = [batch size, units]

            h_4 = F.sigmoid(self.hidden_fc3(h_3))

            # h_4 = [batch size, units]

            h_5 = F.sigmoid(self.hidden_fc4(h_4))

             # h_5 = [batch size, units]

            h_6 = F.sigmoid(self.hidden_fc5(h_5))

            # h_6 = [batch size, units]
        
        
        y_pred = 0 # initialize, WILL be overriden
        if self.activ == 'softmax':
            y_pred = F.softmax(self.output_fc(h_6), dim=1)
        elif self.activ == 'sigmoid':
            y_pred = F.sigmoid(self.output_fc(h_6))


        # y_pred = [batch size, output dim]

        return y_pred, h_6
    
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    
    # Honestly, this may completely invalidate the accuracy, was used since y shape was [1000, 100] and it needed to be 1000
    y = y.argmax(1, keepdim=True)
    # changing top_pred to y_pred and leaving y the save yielded wildly accuracies ()
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()
    
    for (x, y) in tqdm(iterator, desc="Training", leave=False):
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


# Open and Preprocess Files

In [25]:
with open('test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

with open(f'train', 'rb') as f:
    print(f)
    train_dict = pickle.load(f, encoding='bytes')


X_train = train_dict[b'data']
y_train = train_dict[b'coarse_labels']

X_test = test_dict[b'data']
y_test = test_dict[b'coarse_labels']

y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

X_train = np.array(X_train, dtype='float32')
X_train = torch.tensor(X_train)
y_train = np.array(y_train, dtype='float32')
y_train = torch.tensor(y_train)

X_test = np.array(X_test, dtype='float32')
X_test = torch.tensor(X_test)
y_test = np.array(y_test, dtype='float32')
y_test = torch.tensor(y_test)

#y_train = y_train.astype(byte)
#print(y_train)

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')



VALID_RATIO = 0.9

n_train_examples = int(len(X_train) * VALID_RATIO)
n_valid_examples = len(X_train) - n_train_examples

X_train = data.TensorDataset(X_train.unsqueeze(1), y_train)

X_train, valid_data = data.random_split(X_train,
                                           [n_train_examples, n_valid_examples])


#print(test_iterator.dataset)

<_io.BufferedReader name='/Users/noahs/Desktop/College/2024/Spring 2024/CPSC4430/Assignments/Homework2/assignment_2/assignment_2/train'>
x_train shape: torch.Size([50000, 3072])
y_train shape: torch.Size([50000, 100])
50000 train samples
10000 test samples


# Modeling All Possible

In [34]:
# parameters
params = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'activation': ['softmax', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}

#Sourced from itertools.product
def product(*args, repeat=1):
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)


# clear outputLoss.csv
f = open("outputLoss.csv", "w")
f.truncate()
f.close()


EPOCHS = 3

best_valid_loss = float('inf')
isFirst = True

for i in product(params['units'],params['hidden_activations'],params['activation'],params['loss'],params['optimizer'],params['batch_size']):
    #print(i)
    BATCH_SIZE = i[5]


    train_iterator = data.DataLoader(X_train,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

    valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

    test_iterator = data.DataLoader(X_test,
                                batch_size=BATCH_SIZE)


    model = MLP(3072,100, i[0], i[1], i[2])

    optimizer = ''
    #print(f'The model has {count_parameters(model):,} trainable parameters')
    if i[4] == 'adam':
        optimizer = optim.Adam(model.parameters())
    elif i[4] == 'adagrad':
        optimizer = optim.Adagrad(model.parameters())
    
    criterion = ''
    if i[3] == 'mse':
        criterion = nn.MSELoss()
    elif i[3] == 'categorical_crossentropy':
        criterion = nn.CrossEntropyLoss()
    

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    criterion = criterion.to(device)

    train_loss, train_acc = 0,0 # initialize for csv use
    valid_loss, valid_acc = 0,0 
    
    for epoch in trange(EPOCHS):

        start_time = time.monotonic()

        train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'tut1-model.pt')

        end_time = time.monotonic()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        #print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    # Prints useful if we weren't already placing it into csv files
    #print('\t Param: \n\t\tUnits: '+str(i[0]))
    #print('\t\tHidden Activation: '+str(i[1]))
    #print('\t\tActivation: '+str(i[2]))
    #print('\t\tLoss: '+str(i[3]))
    #print('\t\tOptimizer: '+str(i[4]))
    #print('\t\tBatch Size: '+str(i[5]))
    
    # Writes the dataframe
    columns = ['round_epoch','t_accuracy','t_loss','val_accuracy','val_loss','units','hidden_activations','activation','loss','optimizer','batch_size']
    datadf = [[EPOCHS, train_acc, train_loss, valid_acc, valid_loss, i[0], str(i[1]), str(i[2]), str(i[3]), str(i[4]), i[5]]]
    df = pd.DataFrame(datadf, columns=columns)
    if isFirst:
        df.to_csv('outputLoss.csv', mode='a', header=True, index=False)
        isFirst = False
    else:
        df.to_csv('outputLoss.csv', mode='a', header=False, index=False)
        

  0%|          | 0/3 [00:00<?, ?it/s]

Training:   0%|          | 0/45 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Training:   0%|          | 0/45 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Training:   0%|          | 0/23 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Training:   0%|          | 0/23 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

In [35]:
# We can easily see within needing to use pandas, that 
'''
    Units: 120
        Hidden Activation: relu
        Activation: softmax
        Loss: mse
        Optimizer: adam
        Batch Size: 1000
        
    Was the most accurate, with 18%.

'''


df = pd.read_csv('outputLoss.csv')
df = df.sort_values(by=['t_accuracy'], ascending=False)
df

,round_epoch,t_accuracy,t_loss,val_accuracy,val_loss,units,hidden_activations,activation,loss,optimizer,batch_size
0,3,0.180200,0.008956,0.203400,0.008864,120,relu,softmax,mse,adam,1000
32,3,0.164000,0.009014,0.185400,0.008860,240,relu,softmax,mse,adam,1000
33,3,0.135565,0.009201,0.143667,0.009122,240,relu,softmax,mse,adam,2000
1,3,0.100717,0.009428,0.123000,0.009313,120,relu,softmax,mse,adam,2000
17,3,0.051565,0.009503,0.053667,0.009503,120,sigmoid,softmax,mse,adam,2000
...,...,...,...,...,...,...,...,...,...,...,...
54,3,0.047333,4.565795,0.047200,4.566941,240,sigmoid,softmax,categorical_crossentropy,adagrad,1000
41,3,0.024587,0.010000,0.026000,0.010000,240,relu,sigmoid,mse,adam,2000
40,3,0.006556,0.010000,0.005000,0.010000,240,relu,sigmoid,mse,adam,1000
11,3,0.004500,0.020780,0.000000,0.020000,120,relu,sigmoid,mse,adagrad,2000
